<a href="https://colab.research.google.com/github/leah-apking/project3.1/blob/main/PKE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Install & Import Dependencies

In [1]:
pip install git+https://github.com/boudinfl/pke.git

  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-jr8mx6f4
  Running command git clone --filter=blob:none --quiet https://github.com/boudinfl/pke.git /tmp/pip-req-build-jr8mx6f4
  Resolved https://github.com/boudinfl/pke.git to commit 69871ffdb720b83df23684fea53ec8776fd87e63
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.2 MB/s eta 0:00:00
  Created wheel for pke: filename=pke-2.0.0-py3-none-any.whl size=6160628 sha256=ce5de0c9ae30d9fdd0ce04594870bd92334619669f414d6724a885e936c945e5
  Stored in directory: /tmp/pip-ephem-wheel-cache-fazan336/wheels/8c/07/29/6b35bed2aa36e33d77ff3677eb716965ece4d2e56639ad0aab
Successfully built pke


In [2]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 8.6 MB/s eta 0:00:00


In [3]:
!python -m spacy download en_core_web_sm

2023-11-02 13:14:19.135172: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-02 13:14:19.135263: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-02 13:14:19.135325: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-02 13:14:19.149773: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-02 13:14:21.736955: W tensorflow/compiler/

In [4]:
import pandas as pd
import numpy as np
import pke
from transformers import pipeline

## Pull NPS Feedback Data

In [30]:
month=  '09'
year = '23'
nps_path = f"{year}.{month}ClientNPSSurveyFeedback.csv"
nps_data = pd.read_csv(nps_path)

In [31]:
nps = nps_data[['FullName', 'LoanNumber', 'CompanyName', 'NetPromoterLabel', 'Rating',
       'Comments', 'PerformedBy']]
nps0 = nps.rename(columns={'FullName':'ClientName', 'PerformedBy':'TeamMemberName'})
nps1 = nps0.fillna(0)
nps1["Comment"] = nps1.Comments.map(lambda x: 'False' if x == '0' else 'True')

In [32]:
nps2 = nps1.loc[nps1['Comment'] == 'True']
comments = nps2[['LoanNumber', 'Comments']]
comments_df = comments.dropna()
comments.head()

,LoanNumber,Comments
3,1.223571e+09,"Underwriter was wonderful on this file, we wer..."
4,1.223488e+09,"First and foremost, our AE rocks!! And overall..."
5,1.223542e+09,The file went very smoothly until we needed a ...
13,1.523124e+09,Everything was great!
14,1.223571e+09,Super service driven lender


In [33]:
comments['WordCount'] = comments['Comments'].str.count(' ') + 1
nps_short = comments.loc[(comments['WordCount'] > 0) & (comments['WordCount'] < 10)]
nps_short.count()

<ipython-input-33-a644941f0a86>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['WordCount'] = comments['Comments'].str.count(' ') + 1


LoanNumber    262
Comments      262
WordCount     262
dtype: int64

In [34]:
comments['WordCount'] = comments['Comments'].str.count(' ') + 1
nps_med = comments.loc[(comments['WordCount'] >= 10) & (comments['WordCount'] <= 30)]
nps_med.count()

<ipython-input-34-840eeb0e71f6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['WordCount'] = comments['Comments'].str.count(' ') + 1


LoanNumber    274
Comments      274
WordCount     274
dtype: int64

In [35]:
comments['WordCount'] = comments['Comments'].str.count(' ') + 1
nps_long = comments.loc[(comments['WordCount'] > 30) & (comments['WordCount'] <= 400)]
nps_long.count()

<ipython-input-35-906bb5f5c8cb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['WordCount'] = comments['Comments'].str.count(' ') + 1


LoanNumber    220
Comments      220
WordCount     220
dtype: int64

In [36]:
comments['WordCount'] = comments['Comments'].str.count(' ') + 1
nps_xl = comments.loc[comments['WordCount'] > 400]
nps_xl.count()

<ipython-input-36-593fb9c2164a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['WordCount'] = comments['Comments'].str.count(' ') + 1


LoanNumber    3
Comments      3
WordCount     3
dtype: int64

## Extract Sentiment and KeyPhrases from Comments

### "Short" Comments (WordCount < 10)

In [37]:
def get_sentiment(text):

  sentiment_pipeline = pipeline(task = 'sentiment-analysis',
                     model = 'nlptown/bert-base-multilingual-uncased-sentiment',
                     tokenizer = 'nlptown/bert-base-multilingual-uncased-sentiment')

  return sentiment_pipeline(text)

def pke_extract(text):

  extractor = pke.unsupervised.TopicRank()

  extractor.load_document(input=text, language='en')

  extractor.candidate_selection()

  extractor.candidate_weighting()

  return extractor.get_n_best(n=n, stemming=False)

n=2
nps_short['Keywords'] = nps_short['Comments'].apply(pke_extract)

nps_short['Sentiment'] = nps_short['Comments'].apply(get_sentiment)

<ipython-input-37-3ef077a83b70>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_short['Keywords'] = nps_short['Comments'].apply(pke_extract)
<ipython-input-37-3ef077a83b70>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_short['Sentiment'] = nps_short['Comments'].apply(get_sentiment)


In [38]:
nps_short['Keywords'] = nps_short['Keywords'].astype(str)

def clean_keywords(keywords_str):
  keywords_list = eval(keywords_str)
  return [keyword for keyword, _ in keywords_list]

nps_short['Keywords'] = nps_short['Keywords'].apply(clean_keywords)

<ipython-input-38-367733debbc6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_short['Keywords'] = nps_short['Keywords'].astype(str)
<ipython-input-38-367733debbc6>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_short['Keywords'] = nps_short['Keywords'].apply(clean_keywords)


In [39]:
nps_short['Sentiment'] = nps_short['Sentiment'].astype(str)

def clean_sentiment(sentiment_str):
  sentiment_list = eval(sentiment_str)
  return sentiment_list[0]['label'].split(' ')[0]

nps_short['Sentiment'] = nps_short['Sentiment'].apply(clean_sentiment)

<ipython-input-39-e2e120fc2df2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_short['Sentiment'] = nps_short['Sentiment'].astype(str)
<ipython-input-39-e2e120fc2df2>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_short['Sentiment'] = nps_short['Sentiment'].apply(clean_sentiment)


In [40]:
# nps_short = nps_short.drop(columns={'Keyword', 'Keyword1'})

nps_short.head(15)

,LoanNumber,Comments,WordCount,Keywords,Sentiment
13,1.523124e+09,Everything was great!,3,[great],5
14,1.223571e+09,Super service driven lender,4,"[lender, super service]",5
15,1.223516e+09,"excellent service, as always!",4,[excellent service],5
25,1.223540e+09,It was awesome,3,[awesome],5
45,1.223225e+09,You are great,3,[great],5
47,1.223562e+09,Everything went smooth and quick! Thanks!,6,"[quick, smooth]",5
48,1.223522e+09,best lender ever !!!! LFG!!!!,5,"[best lender, lfg]",5
51,1.223554e+09,The best in the husbands down!!,6,"[best, husbands]",5
75,1.223577e+09,Omar Noja should be employee of the month!,8,"[employee, month]",5
81,1.223425e+09,Helpful and communicative,3,"[communicative, helpful]",5


### Medium Comments (WordCount 10-30)

In [41]:
# def pke_extract5(text):

#   extractor = pke.unsupervised.TopicRank()

#   extractor.load_document(input=text, language='en')

#   extractor.candidate_selection()

#   extractor.candidate_weighting()

#   return extractor.get_n_best(n=5, stemming=False)

n=4
nps_med['Keywords'] = nps_med['Comments'].apply(pke_extract)

nps_med['Sentiment'] = nps_med['Comments'].apply(get_sentiment)

<ipython-input-41-f88018c8d639>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_med['Keywords'] = nps_med['Comments'].apply(pke_extract)
<ipython-input-41-f88018c8d639>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_med['Sentiment'] = nps_med['Comments'].apply(get_sentiment)


In [42]:
nps_med['Keywords'] = nps_med['Keywords'].astype(str)
nps_med['Sentiment'] = nps_med['Sentiment'].astype(str)

nps_med['Keywords'] = nps_med['Keywords'].apply(clean_keywords)
nps_med['Sentiment'] = nps_med['Sentiment'].apply(clean_sentiment)

<ipython-input-42-ce786f379d17>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_med['Keywords'] = nps_med['Keywords'].astype(str)
<ipython-input-42-ce786f379d17>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_med['Sentiment'] = nps_med['Sentiment'].astype(str)
<ipython-input-42-ce786f379d17>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

In [43]:
# #split column of lists into two new columns
# split = pd.DataFrame(nps_med['Keywords'].to_list(), columns = ['Keyword1', 'Keyword2', 'Keyword3'])

# #join split columns back to original DataFrame
# nps_med = pd.concat([nps_med, split], axis=1)

In [44]:
nps_med.head()

,LoanNumber,Comments,WordCount,Keywords,Sentiment
3,1.223571e+09,"Underwriter was wonderful on this file, we wer...",13,"[wonderful, underwriter, file, days]",5
4,1.223488e+09,"First and foremost, our AE rocks!! And overall...",21,"[course, underwriting, great, things]",5
35,1.223528e+09,You guys are always the best ! Great job team.,10,"[best, great job team, guys]",5
55,1.223571e+09,"As always, Olja and her band of magicians come...",18,"[band, magicians, quick turnaround cashout, th...",5
74,1.223511e+09,As always a pleasure working with UWM! Demarco...,29,"[uwm, demarco dukho, grateful, borrower]",5


### Long Comments (WordCount 30-400)

In [45]:
# def get_sentiment(text):

#   sentiment_pipeline = pipeline(task = 'sentiment-analysis',
#                      model = 'nlptown/bert-base-multilingual-uncased-sentiment',
#                      tokenizer = 'nlptown/bert-base-multilingual-uncased-sentiment')

#   return sentiment_pipeline(text)

# def pke_extract8(text):

#   extractor = pke.unsupervised.TopicRank()

#   extractor.load_document(input=text, language='en')

#   extractor.candidate_selection()

#   extractor.candidate_weighting()

#   return extractor.get_n_best(n=8, stemming=False)

n = 8
nps_long['Keywords'] = nps_long['Comments'].apply(pke_extract)

nps_long['Sentiment'] = nps_long['Comments'].apply(get_sentiment)

<ipython-input-45-7479e77bcc86>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_long['Keywords'] = nps_long['Comments'].apply(pke_extract)
<ipython-input-45-7479e77bcc86>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_long['Sentiment'] = nps_long['Comments'].apply(get_sentiment)


In [46]:
nps_long['Keywords'] = nps_long['Keywords'].astype(str)
nps_long['Sentiment'] = nps_long['Sentiment'].astype(str)

nps_long['Keywords'] = nps_long['Keywords'].apply(clean_keywords)
nps_long['Sentiment'] = nps_long['Sentiment'].apply(clean_sentiment)

<ipython-input-46-3aba88d1555d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_long['Keywords'] = nps_long['Keywords'].astype(str)
<ipython-input-46-3aba88d1555d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nps_long['Sentiment'] = nps_long['Sentiment'].astype(str)
<ipython-input-46-3aba88d1555d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

In [47]:
# #split column of lists into two new columns
# split = pd.DataFrame(nps_long['Keywords'].to_list(), columns = ['Keyword1', 'Keyword2', 'Keyword3', 'Keyword4', 'Keyword5'])

# #join split columns back to original DataFrame
# nps_long = pd.concat([nps_long, split], axis=1)

## Complete DataFrame

In [48]:

nps_short.tail()

,LoanNumber,Comments,WordCount,Keywords,Sentiment
2131,230987816.0,Great experience,2,[great experience],5
2133,230933495.0,Always a pleasure working with your fantastic ...,8,"[fantastic team, pleasure]",5
2146,230969384.0,Great easy experience using the UWM portal.,7,"[great easy experience, uwm portal]",5
2149,230924098.0,Excellent!,1,[excellent],5
2164,230941572.0,Great platform to work with.,5,[great platform],5


In [49]:
concat1 = pd.concat([nps_short, nps_med])
concat2 = pd.concat([concat1, nps_long])
concat2.tail()

,LoanNumber,Comments,WordCount,Keywords,Sentiment
2135,230994793.0,Underwriter II Myrthi Posa did not respond to ...,193,"[condition, loans, additional touches, clarifi...",2
2144,230971947.0,My experience was excellent as always. Being a...,48,"[able, difference, excellent, world, experienc...",5
2153,230964969.0,UWM do a fantastic job! At this moment UW are ...,38,"[uwm, bank statement programs, turn time, mome...",5
2156,230986449.0,Out of the loans I've been able to complete wi...,53,"[loans, able, uw review times, uwm, timetable,...",3
2161,230910117.0,I have worked with UWM for years but for the p...,60,"[proper help, years, important, uwm, miss cond...",2


In [50]:
merged = pd.merge(concat2, nps1, on='LoanNumber')
merged1 = merged.drop(columns={'Comments_y', 'Comment', 'NetPromoterLabel'})
merged2 = merged1.rename(columns={'Comments_x': 'Comments'})
merged2.tail()

,LoanNumber,Comments,WordCount,Keywords,Sentiment,ClientName,CompanyName,Rating,TeamMemberName
792,230994793.0,Underwriter II Myrthi Posa did not respond to ...,193,"[condition, loans, additional touches, clarifi...",2,Susan Thomas Owen,"Centennial Mortgage, Inc.",5,0
793,230971947.0,My experience was excellent as always. Being a...,48,"[able, difference, excellent, world, experienc...",5,Terri Sullivan,Haus Capital Corporation (Main),10,0
794,230964969.0,UWM do a fantastic job! At this moment UW are ...,38,"[uwm, bank statement programs, turn time, mome...",5,Umberto Filesari,Mortgage and Commercial Lending Inc,10,0
795,230986449.0,Out of the loans I've been able to complete wi...,53,"[loans, able, uw review times, uwm, timetable,...",3,Walter Henderson,Edge Home Finance Corporation I,7,0
796,230910117.0,I have worked with UWM for years but for the p...,60,"[proper help, years, important, uwm, miss cond...",2,Yanet Perez,"Florida Homeowner Advisors, Inc.",3,0


In [51]:
from google.colab import files
merged2.to_csv(f'CommentsProcessed_20{year}.{month}.csv')
files.download(f'CommentsProcessed_20{year}.{month}.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [52]:
# final_df = merged2.groupby(merged2['LoanNumber']).first()

In [53]:
# nps_short = nps_short.drop(columns={'Keywords'})
# nps_med = nps_med.drop(columns={'Keywords'})
# nps_long = nps_long.drop(columns={'Keywords'})

In [54]:
# from google.colab import files
# nps_short.to_csv('CommentsProcessed4.csv', encoding = 'utf-8-sig')
# files.download('CommentsProcessed4.csv')

# nps_med.to_csv('CommentsProcessed2.csv', encoding = 'utf-8-sig')
# files.download('CommentsProcessed2.csv')

# nps_long.to_csv('CommentsProcessed3.csv', encoding = 'utf-8-sig')
# files.download('CommentsProcessed3.csv')